In [18]:
import pickle
import os
import data_analysis.preprocessor_end as pre_processer
import numpy as np
from shutil import copyfile

In [2]:
def out_pickle(pickle_path, pick_name, variable_name):
    with open(pickle_path + pick_name + ".pkl", "wb") as pkl:
        pickle.dump(variable_name, pkl)

def in_pickle(pickle_path, pick_name):
    with open(pickle_path + pick_name + ".pkl", "rb") as pkl:
        return pickle.load(pkl)

In [3]:
word2vec_model = in_pickle("C:/Users/Abdo/Project/pre-trained word2vec models-compiled/",
                               "word2vecModel_glove_6b_300d")
vocabulary = set(word2vec_model.wv.vocab)

C:\Users\Abdo\PycharmProjects\final_project_1\venv\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


C:\Users\Abdo\PycharmProjects\final_project_1\venv\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# returns the average, n dimensioned, vector embedding of every word in a given document
# it sums all the vector values of every word in the document and then divides them by their count
# if a given word is not in the vocabulary of the model, it ignores it
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

In [3]:
# utf8: true if the file is a .txt file, false otherwise
def test_classifier_on_outside_data(classifer, directory_path, txt_file_name, utf8):
    txt_data_tokens = pre_processer.preprocess_file(directory_path, txt_file_name, utf8)
    #print(txt_data_tokens)
    text_vector_representation = average_word_vectors(txt_data_tokens, word2vec_model, vocabulary, 300)
    
    prediction = classifer.predict([text_vector_representation])
    
    return prediction

def get_metrices(t_p, t_n, f_p, f_n):
    precision = t_p / (t_p + f_p)
    accuracy = (t_p + t_n) / (t_p + f_p + f_n + t_n)
    recall = t_p / (t_p + f_n)
    
    return (precision, accuracy, recall, t_p, f_p, t_n, f_n)

# takes the prediction dict and returns its confusion matrix
def get_confusion_matrix(predictions_dict):
    t_p = 0
    f_p = 0
    t_n = 0
    f_n = 0
    for label in predictions_dict:
        for moviename_prediction in predictions_dict[label]:
            prediction = moviename_prediction[1]
            # if the label is 1
            if(label == "1"):
                # the label is 1 and we predicted 1
                if prediction == 1:
                    t_p += 1
                # the label is 1 and we predicted 0
                elif prediction == 0:
                    f_n += 1
            # if the label is 0
            elif label == "0":
                # the label is 0 and we predicted 1 
                if prediction == 1:
                    f_p += 1
                # the label is 0 and we predicted 0
                elif prediction == 0:
                    t_n += 1
    return t_p, t_n, f_p, f_n

In [6]:
clf = in_pickle("C:/Users/Abdo/Project/pickle_files/","svm_final_classifier")

In [18]:
# key : label
# value [ (movie_name, prediction),...  ]
predictions_dict = { "0":[], "1":[], "not_labeled":[]}

In [19]:
direcotry_path = "C:/Users/Abdo/Project/test_data/books_txt/"
direcotry_folders = os.listdir(direcotry_path)

In [45]:
try:
    prediction = test_classifier_on_outside_data(clf,
                            direcotry_path,
                            direcotry_folders[10],False)

except:
 prediction = test_classifier_on_outside_data(clf,
                            direcotry_path,
                            direcotry_folders[10],True)

print(prediction)

AB-5475 Family Peepers by David Crane.txt


[1]


In [20]:
for folder in direcotry_folders:
    movies = os.listdir(direcotry_path + folder + "/")
    for movie_name in movies:
        try:
            movie_prediction = test_classifier_on_outside_data(clf, direcotry_path + folder + "/", movie_name, False)
        except:
            print("\n")
            movie_prediction = test_classifier_on_outside_data(clf, direcotry_path + folder + "/", movie_name, True)
            print("\n")

        predictions_dict[folder].append( (movie_name, movie_prediction) )

5042 Voodoo In Haiti by Aubrey Wilder.txt


A Night To Remember by Argus.txt


AB-1609 House of Dark Pleasure by Gloria Day.txt


AB-1676 Beasts Are Better by Richard Long.txt


AB-1680 The Dog Ballers by Don Russell.txt


...



the-great-gatsby.txt


The-House-of-the-Dead.txt


The-House-of-the-Dead.txt




The-Idiot.txt


The-Peasant-Marey.txt


The-Peasant-Marey.txt


To Build a Fire by Jack London a Free Audio Book - English (auto-generated).txt


[Fantasy Tale from the Author of Dracula] The Castle of the King (Bram Stoker Audiobook) - English (auto-generated).txt


In [21]:
out_pickle("C:/Users/Abdo/Project/pickle_files/","predictions_dict_books_txt", predictions_dict)

In [4]:
movies_txt_dict = in_pickle("C:/Users/Abdo/Project/pickle_files/","predictions_dict_movies_txt")
books_txt_dict = in_pickle("C:/Users/Abdo/Project/pickle_files/","predictions_dict_books_txt")

In [5]:
books_txt_dict.pop('not_labeled', None)

[('11 TRUE SCARY STORIES From Reddit [MEGA COLLABORATION #3] - English (auto-generated).txt',
  array([0])),
 ('11 TRUE SCARY STORIES From Reddit [MEGA COLLABORATION #4] - English (auto-generated).txt',
  array([1])),
 ('14 Horror Stories Animated (Compilation of 2015) - English (auto-generated).txt',
  array([1])),
 ('14 More Horror Stories Animated (Compilation of 2016) - English (auto-generated).txt',
  array([1])),
 ('16 True Scary Stories [Compilation Vol.14] - English (auto-generated).txt',
  array([1])),
 ('5 Horror Stories Animated (Compilation of Dec. 2018) - English (auto-generated).txt',
  array([1])),
 ('A novel by - Jeffrey Carver ( PDFDrive.com ).txt', array([0])),
 ('A Sherlock Holmes Novel_ The Sign of the Four Audiobook - English (United Kingdom).txt',
  array([1])),
 ('A TALE OF TWO CITIES by Charles Dickens - FULL AudioBook _ Greatest AudioBooks (Book 1 of 3) V2 - English (auto-generated).txt',
  array([1])),
 ('Audiobooks - The Little Prince - Antoine de Saint-Exupé

In [6]:
# key: movies_txt_dict, books_txt_dict
# value: (precision, accuracy, recall, t_p, f_p, t_n, f_n)
test_results_dict = { }

In [7]:
# get the confusion matrix for each one of the cases
movies_txt_confusion_matrix = get_confusion_matrix(movies_txt_dict)
books_txt_confusion_matrix = get_confusion_matrix(books_txt_dict)

In [8]:
# the movies text case
movies_txt_metrics = get_metrices(movies_txt_confusion_matrix[0],
                                  movies_txt_confusion_matrix[1],
                                  movies_txt_confusion_matrix[2],
                                  movies_txt_confusion_matrix[3])

In [9]:
# the books text case
books_txt_metrics = get_metrices(books_txt_confusion_matrix[0],
                                  books_txt_confusion_matrix[1],
                                  books_txt_confusion_matrix[2],
                                  books_txt_confusion_matrix[3])

In [10]:
test_results_dict["movies_txt_dict"] = movies_txt_metrics
test_results_dict["books_txt_dict"] = books_txt_metrics

In [29]:
out_pickle("C:/Users/Abdo/Project/pickle_files/","test_results_dict", test_results_dict)

In [8]:
test_results_dict = in_pickle("C:/Users/Abdo/Project/pickle_files/","test_results_dict")

In [11]:
print(test_results_dict["books_txt_dict"])

(0.36681222707423583, 0.4128787878787879, 0.8936170212765957, 84, 145, 25, 10)


In [12]:
books_txt_dict = in_pickle("C:/Users/Abdo/Project/pickle_files/","predictions_dict_books_txt")

In [13]:
# key: predicted label
# value: [book1_name, ..... ]
non_labeled_books_dict = {0: [], 1:[]}

In [14]:
for book_name_prediction_pair in books_txt_dict["not_labeled"]:
    book_name = book_name_prediction_pair[0]
    prediction = book_name_prediction_pair[1][0]
    
    non_labeled_books_dict[prediction].append(book_name)

In [20]:
for prediction in non_labeled_books_dict:
    for book in non_labeled_books_dict[prediction]:
        os.rename("C:/Users/Abdo/Project/test_data/books_txt/not_labeled/"+book,
                  "C:/Users/Abdo/Project/test_data/books_txt/not_labeled/"+str(prediction)+"_"+book)